In [1]:
cd ..

/mnt/c/mcs-formalizations


In [2]:
from rdflib import Graph, URIRef
from mcs_formalizations.namespace import bind_namespaces, MCS, RDF
from mcs_formalizations.helpers import uri_base

### Question: True/False - A parking garage is larger than an automobile.

##### Prepare the Graph

In [3]:
from mcs_formalizations.reasoners.sparql_reasoner import SparqlReasoner
parking_garage = uri_base("parking_garage")
automobile = uri_base("automobile")
vehicle = uri_base("vehicle")

##### Minimum Knowledge Assumed:
- a parking garage contains vehicles
- an automobile is a type of vehicle

In [4]:
reasoner = SparqlReasoner()

reasoner.add_knowledge(
    (parking_garage, MCS.contains, vehicle),
    (vehicle, RDF.type, automobile),
)

Graph after adding knowledge:
@prefix mcs: <http://purl.org/twc/mcs/> .

<mcs:object:parking_garage> mcs:contains <mcs:object:vehicle> .

<mcs:object:vehicle> a <mcs:object:automobile> .




#### Minimum rules assumed:

- a container is larger than the things it contains

In [5]:
reasoner.apply_rules(
    "isLargerThan",
)

TypeError: transitive_objects() missing 1 required positional argument: 'property'

In [ ]:
print("Is a parking garage larger than an automobile?")
print(reasoner.ask((uri_base(parking_garage), "mcs:isLargerThan", uri_base(automobile))))